# STEP 1: Cleaning the data

In [1]:
import sentencepiece as spm
import os
from tokenizers import SentencePieceBPETokenizer
import string
import re
import sys
from unicodedata import normalize
import os
from pickle import dump

**Load data**

In [20]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

**Split data into sentences**

In [3]:
# split a loaded document into sentences
def to_sentences(doc):
    return doc.strip().split('\n')

**Clean the data**

*Tibetan*

* Remove shey "།" (end of the line punctuation) 
* Remove any numbers 
* Remove any charcter that isn't Tibetan alphabet example  ༼༄༸ or non-Tibetan characters 
* There is no upper or lower case in Tibetan so no need to normalize 
* Make sure that there is no space between words (seperated by tsek "་")
* Remove words that contain non Tibetan alphabets example: བས྄ྟན༼་


**Create Tibetan unicode array for easy access**

*Tibetan Unicode Array:*

* Tibetan Vowel : (ུ): 3956 ( ི) : 3954 ( ེ) : 3962  ( ོ) : 3964
* Consonants : 3904 - 3946 
* Subjoined Consonants : 3984 - 4028 
* Numbers : 3872 - 3881 
* punctuation: Tsek (་) : 3851 ; shey (།) : 3853 


In [4]:
tib_unicode = []
# adding consonants 
for i in range(3904, 3947):
    tib_unicode.append(i)
# adding subjoined consonants 
for i in range(3984, 4029):
    tib_unicode.append(i)
# adding numbers 
for i in range(3872, 3881):
    tib_unicode.append(i)
# adding punctuations 
tib_unicode.append(3851)
tib_unicode.append(3853)
# adding vowels
tib_unicode.append(3956)
tib_unicode.append(3954)
tib_unicode.append(3962)
tib_unicode.append(3964)

tib_str = "" # Contains all Tibetan alphabets, numbers, and special characters.
tib_alph_str = "" # Contains only Tibetan alphabets
tib_num = "" # Contains only Tibetan numbers 

for i in range(3904, 3947):
    tib_str += chr(i)
    tib_alph_str += chr(i)
# adding subjoined consonants 
for i in range(3984, 4029):
    tib_str += chr(i)
    tib_alph_str += chr(i)
# adding numbers 
for i in range(3872, 3881):
    tib_str += chr(i)
    tib_num += chr(i)
# adding punctuations 
tib_str += chr(3851)
tib_str += chr(3853)
# adding vowels
tib_str += chr(3956)
tib_str += chr(3954)
tib_str += chr(3962)
tib_str += chr(3964)

tib_alph_str += chr(3956)
tib_alph_str += chr(3954)
tib_alph_str += chr(3962)
tib_alph_str += chr(3964)

In [5]:

# checks if the word contains non Tibetan alphabets
def isalpha(word):
    for w in word:
        if w not in tib_alph_str:
            return False
    return True

# clean a list of lines (Tibetan)
def clean_lines_bo(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(tib_str))

    for line in lines:
        
        #  remove strings between [] that was not translated into English (this is for this specific data)
        line = re.sub("[\(\[].*?[\)\]]", "", line)

        # tokenize on tsek and shek
        line = re.split("་|།", line)

        # remove non-printable chars form each token
        line = [re_print.sub('', w) for w in line]

        # remove tokens with numbers in them
        line = [word for word in line if isalpha(word)]

        line = '་'.join(line)

        # remove any empty line or white spaces at the end of the line
#         if line.rstrip():
            
        # store as string (removed shek)
        cleaned.append(line)

    return cleaned


*English*

* Remove punctuation 
* Remove any numbers 
* Remove any character that is not an English alphabet 
* Normalize everything to lower letters 
* Remove words that contain non English alphabets 

In [6]:
# clean a list of lines (English)
def clean_lines_en(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))

    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)

    for line in lines:
        # normalize unicode characters
        line = normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('UTF-8')

        # tokenize on white space
        line = line.split()

        # convert to lower case
        line = [word.lower() for word in line]

        # remove punctuation from each token
        line = [word.translate(table) for word in line]

        # remove non-printable chars form each token
        line = [re_print.sub('', w) for w in line]
        
        # remove tokens with numbers in them
        line = [word for word in line if word.isalpha()]
    
        # # store as string
#         if not line == []:    
        cleaned.append(' '.join(line))

    return cleaned

**Save the clean sentences to a file** 

In [7]:
# save a list of clean sentences to file
def save_clean_sentences(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as filehandle:
        filehandle.writelines("%s\n" % sentence for sentence in sentences)

    print('Saved: %s' % filename)

In [18]:
def save_clean_sentences_binary(sentences, folder, filename):
    if not os.path.exists(folder):
        os.makedirs(folder)
    dump(sentences, open(filename, 'wb'))

    print('Saved: %s' % filename)

**Get information on the shortest and longest sentences in the two data**

*English*

In [9]:
# shortest and longest sentence lengths
def sentence_lengths_en(sentences):
	lengths = [len(s.split()) for s in sentences]
	return min(lengths), max(lengths)

*Tibetan*

In [10]:
def sentence_lengths_bo(sentences):
	lengths = [len(s.split("་")) for s in sentences]
	return min(lengths), max(lengths)

In [11]:
if __name__ == '__main__':
    bo_text = "../data/bo.txt"
    en_text = "../data/en.txt"
    clean_bo = "../data/train.bo"
    clean_en = "../data/train.en"

    # Tibetan
    doc = load_doc(bo_text)
    sentences = to_sentences(doc)
    sentences = clean_lines_bo(sentences)
    minlen, maxlen = sentence_lengths_bo(sentences)
    print('Tibetan data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
    save_clean_sentences(sentences, clean_bo) 
    # spot check
    for i in range(5):
        print(sentences[i])
    print()
    # English
    doc = load_doc(en_text)
    sentences = to_sentences(doc)
    sentences = clean_lines_en(sentences)
    minlen, maxlen = sentence_lengths_en(sentences)
    print('English data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))

    save_clean_sentences(sentences, clean_en)
    
    # spot check
    for i in range(5):
        print(sentences[i])


Tibetan data: sentences=106872, min=1, max=894
Saved: ../data/train.bo
རྒྱལ་པོ་ཞེས་བྱ་བས་རྒྱལ་སྲིད་འབྱོར་པ་རྒྱས་པ་བདེ་བ་ལོ་ལེགས་པ་སྐྱེ་བོ་དང་མི་མང་པོས་གང་བ་བྱེད་དུ་བཅུག་གོ་
དེས་དཔུང་གི་ཚོགས་ཡན་ལག་བཞི་པ་གླང་པོ་ཆེ་པའི་ཚོགས་དང་རྟ་པའི་ཚོགས་དང་ཤིང་རྟ་པའི་ཚོགས་དང་དཔུང་བུ་ཆུང་གི་ཚོགས་གོ་བསྐོན་ཏེ་ཡུལ་མ་ག་དཧའི་རྒྱལ་པོའི་ཁ་བ་མ་གཏོགས་པ་བཅོམ་ནས་ཕྱིར་ལྡོག་པར་བྱེད་དོ་
སུམ་ཅུ་རྩ་གསུམ་པའི་ལྷ་རྣམས་ཀྱི་ཁ་དོག་གི་མཐུ་བས་ལྷག་པའི་སྣང་བ་རྒྱ་ཆེན་པོས་ཁྱབ་པར་གྱུར་འཇིག་རྟེན་གྱི་འཇིག་རྟེན་གྱི་བར་གང་ན་ཉི་མ་དང་ཟླ་བ་འདི་ལྟར་རྫུ་འཕྲུལ་ཆེ་བ་འདི་ལྟར་མཐུ་ཆེ་བ་འདི་གཉིས་ཀྱི་འོད་དག་ཉམས་སུ་མི་མྱོང་བའི་མུན་པ་མུན་ནག་མུན་པར་བྱེད་པས་གནག་པར་གྱུར་པ་གང་དག་ཡིན་པ་དེ་དག་ཀྱང་དེའི་ཚེ་ན་སྣང་བ་རྒྱ་ཆེན་པོས་ཁྱབ་པར་གྱུར་ནས་སེམས་ཅན་གང་དག་དེར་སྐྱེས་པ་དག་གིས་རང་གི་ལག་པ་བརྐྱང་བ་ཡང་མི་མཐོང་བ་དེ་དག་གིས་ཀྱང་འོད་དེས་སེམས་ཅན་གཅིག་གིས་གཅིག་མཐོང་ནསཤེས་ལྡན་དག་སེམས་ཅན་གཞན་ཡང་འདིར་སྐྱེས་སོ་་ཤེས་ལྡན་དག་སེམས་ཅན་གཞན་ཡང་འདིར་སྐྱེས་སོ་ཞེས་ཤེས་པར་གྱུར་ཏོ་
མ་མ་བརྒྱད་པོ་པང་ན་འཚོ་བའི་མ་མ་གཉིས་དང་ནུ་མ་སྣུན་པའི་མ་མ་གཉིས་དང་དྲི་མ་འཕྱི་བའི་མ་མ་གཉིས་དང་རྩེ་འགྲོགས་ཀྱི་

# Step 2 : Tokenize the training data 

**Generate a Tibetan tokenizer using sentencepiece and monolingual Tibetan data**

Using 32000 vocabulary size .. just because (also it was used by the author who wrote sentencepiece)

**Model Training** 

*Tibetan*

In [12]:
spm.SentencePieceTrainer.train(
    input='../data/boTokenData.txt', 
    model_prefix='bo', 
    vocab_size=32000, 
    pad_id = 3
)
# sp = spm.SentencePieceProcessor(model_file='train.model')
# print(sp.encode(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་'], out_type=str))

*English*

In [13]:
spm.SentencePieceTrainer.train(
    input='../data/enTokenData.txt', 
    model_prefix='en', 
    vocab_size=25000, 
    pad_id = 3
)

**Segmentation**

Tibetan Segmentation Example 

In [14]:
sp = spm.SentencePieceProcessor(model_file='bo.model')
print(sp.encode(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་'], out_type=str))
print(sp.encode(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་', 'བཀ྄ྲ་ཤིས་བདེ་ལེགས།'], out_type=int))
print(sp.decode([4149, 306, 6, 245, 4660, 748]))
print(sp.decode(['▁ངའི་', 'མིང་', 'ལ་', 'བསྟན་', 'སྒྲོལ་མ་', 'ཟེར་']))
sp.get_piece_size()


[['▁ངའི་', 'མིང་ལ་', 'བསྟན་', 'སྒྲོལ་མ་', 'ཟེར་']]
[[3645, 18003, 531, 6258, 2155], [5, 3334, 0, 6082, 4, 6751, 1031, 2262, 1962, 0]]
བྲག་སྐུ་དང་ དེའི་ཚེ་མུ་སྟེགས་ཅན་ལོངས་སྤྱོད་
ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་


32000

English Segmentation example

In [15]:
sp = spm.SentencePieceProcessor(model_file='en.model')
print(sp.encode(["My name isn't Tenzin Dolma Gyalpo"], out_type=str))
print(sp.encode(['My name is Tenzin Dolma Gyalpo', 'Hello'], out_type=int))
print(sp.decode([[8803, 180, 12, 5519, 15171, 17894], [887, 21491]]))
sp.get_piece_size()

[['▁My', '▁name', '▁is', 'n', "'", 't', '▁Tenzin', '▁Dolma', '▁Gyalpo']]
[[8804, 181, 13, 5520, 15172, 17895], [888, 21492]]
['offices know  wrongdoing verdictukje', 'universallina']


25000

**Tokenizing training data**

*Tibetan*

In [21]:
sp = spm.SentencePieceProcessor(model_file='bo.model')
doc = load_doc("../data/train.bo")
sentences = to_sentences(doc)
bo_token = sp.encode(sentences, out_type=str)
save_clean_sentences_binary(bo_token, "../data", "../data/tokenized.bo")
# spot check
for i in range(5):
    print(bo_token[i])

Saved: ../data/tokenized.bo
['▁', 'རྒྱལ་པོ་ཞེས་བྱ་བ', 'ས་', 'རྒྱལ་སྲིད་', 'འབྱོར་པ་རྒྱས་པ་', 'བདེ་བ་ལོ་ལེགས་པ་', 'སྐྱེ་བོ་དང་', 'མི་མང་པོས་གང་བ་', 'བྱེད་དུ་བཅུག་གོ་']
['▁དེས་', 'དཔུང་གི་ཚོགས་', 'ཡན་ལག་བཞི་པ་', 'གླང་པོ་ཆེ་', 'པའི་', 'ཚོགས་དང་', 'རྟ', '་', 'པའི་', 'ཚོགས་དང་', 'ཤིང་རྟ་', 'པའི་', 'ཚོགས་དང་', 'དཔུང་བུ་ཆུང་གི', '་', 'ཚོགས་', 'གོ་བསྐོན་ཏེ་', 'ཡུལ་', 'མ་', 'ག་', 'ད', 'ཧ', 'འི་', 'རྒྱལ་པོའི་', 'ཁ་བ་', 'མ་གཏོགས་པ་', 'བཅོམ་ནས་', 'ཕྱིར་ལྡོག་པ', 'ར་བྱེད་དོ་']
['▁', 'སུམ་ཅུ་རྩ་གསུམ་པ', 'འི་ལྷ་རྣམས་', 'ཀྱི་', 'ཁ་དོག་', 'གི་', 'མཐུ་', 'བས་', 'ལྷག་པའི་', 'སྣང་བ་རྒྱ་ཆེན་པོ', 'ས་ཁྱབ་པར་གྱུར་', 'འཇིག་རྟེན་གྱི་', 'འཇིག་རྟེན་གྱི་', 'བར་', 'གང་ན་', 'ཉི་མ་དང་ཟླ་བ་', 'འདི་ལྟར་', 'རྫུ་འཕྲུལ་ཆེ་བ་', 'འདི་ལྟར་', 'མཐུ་ཆེ་བ་', 'འདི་', 'གཉིས་ཀྱི་', 'འོད་', 'དག་', 'ཉམས་སུ་', 'མི་', 'མྱོང་བ', 'འི་', 'མུན་པ་མུན་ནག་', 'མུན་པར་', 'བྱེད་པས་', 'གནག་', 'པར་གྱུར་པ་', 'གང་དག་ཡིན་པ་', 'དེ་དག་ཀྱང་', 'དེའི་ཚེ་ན་', 'སྣང་བ་རྒྱ་ཆེན་པོ', 'ས་ཁྱབ་པར་གྱུར་', 'ནས་', 'སེམས་ཅན་གང་དག་', 'དེར་སྐྱེས་པ', '་དག་གིས་', 'རང་གི་',

*English*

In [22]:
sp = spm.SentencePieceProcessor(model_file='en.model')
doc = load_doc("../data/train.en")
sentences = to_sentences(doc)
en_token = sp.encode(sentences, out_type=str)
save_clean_sentences_binary(en_token, "../data", "../data/tokenized.en")
# spot check
for i in range(5):
    print(en_token[i])

Saved: ../data/tokenized.en
['▁under', '▁his', '▁rule', '▁the', '▁kingdom', '▁prosper', 'ed', '▁and', '▁thrive', 'd', '▁crops', '▁were', '▁boun', 'tiful', '▁and', '▁the', '▁land', '▁teem', 'ed', '▁with', '▁animals', '▁and', '▁people']
['▁he', '▁called', '▁up', '▁the', '▁four', '▁branch', 'es', '▁of', '▁his', '▁armed', '▁forces', 'the', '▁elephant', '▁corps', '▁the', '▁cavalry', '▁the', '▁charioteer', '▁corps', '▁and', '▁the', '▁infantry', 'and', '▁laid', '▁waste', '▁to', '▁all', '▁of', '▁magadha', '▁save', '▁rajagrha', '▁before', '▁return', 'ing']
['▁bath', 'ed', '▁in', '▁a', '▁vast', '▁light', '▁more', '▁luminous', '▁than', '▁the', '▁glow', '▁of', '▁the', '▁gods', '▁of', '▁the', '▁thirty', 'three', '▁so', '▁great', '▁was', '▁this', '▁miraculous', '▁manifestation', '▁it', '▁was', '▁as', '▁if', '▁the', '▁sun', '▁and', '▁moon', '▁', 'shone', '▁in', '▁the', '▁', 'gulf', '▁bet', 'ween', '▁worlds', '▁so', '▁great', '▁was', '▁its', '▁strength', '▁that', '▁darkness', '▁everywhere', '▁even', '